In [1]:
import os
from glob import glob

import earthpy as et
import earthpy.earthexplorer as etee
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import numpy as np
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import shutil

data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
chi_dir = os.path.join(data_dir, 'chicago-neigborhoods')
ndvi_dir = os.path.join(data_dir, 'chicago-green-space','processed')

for a_dir in [chi_dir, ndvi_dir]:
    if not os.path.exists(chi_dir):
        os.makedirs(chi_dir)

In [10]:
chi_url = ('https://afd.calpoly.edu/facilities/campus-maps/gis-data/files/Parcels.zip')
chi_url
chi_gdf = gpd.read_file(chi_url)
names = ['hill', 'Name2', 'Name3','name4','name5','name6','core','name8','name9','name10','name11','name12']
chi_gdf['name'] = names
chi_gdf

neigh_gdf = (chi_gdf.set_index('name').loc[['core','hill']])
neigh_gdf

,geometry
name,
core,"POLYGON ((5766714.681 2312729.400, 5766335.978..."
hill,"POLYGON ((5777299.703 2315667.808, 5777280.201..."


In [11]:
def download_neighborhood_data(name, geometry, start, end):
    """
Download NAIP raster for a given geometry, start date, and end date
    
    Downloads data from the National Agricoltural Impagery Program (NAIP)
    given a spatial and temporal extent.
    National Agricultural Imagery Program (NAIP). (2023). 
    Multispectral Imagery. 

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    print(f'\nNeighborhood name: {name}')
    bbox = etee.BBox(*geometry.bounds)
    print(f'Lower left x-coordinate:{bbox.llx}')
    naip_downloader = etee.EarthExplorerDownloader(
        dataset="NAIP", 
        label=name.lower().replace(' ','-'), 
        bbox=bbox,
        start=start, 
        end=end,
        store_credential=True)
    naip_downloader.submit_download_request()
    naip_downloader.download(override=False)
    return naip_downloader

ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')
if os.path.exists(ndvi_stats_path):
  print('Reading in NDVI statistics file...')
  ndvi_stats_df = pd.read_csv(ndvi_stats_path)
else: 
   print('NDVI stats file does not exist...')
   ndvi_stats_df = pd.DataFrame()

for neighborhood_name, details in neigh_gdf.iterrows():
  if neighborhood_name in ndvi_stats_df.index:   
    print('Neighborhood stats have been calculated. skipping')
    continue

  downloader = download_neighborhood_data(
   neighborhood_name, details.geometry, '2021-01-01','2021-12-31')

NDVI stats file does not exist...

Neighborhood name: core
Lower left x-coordinate:5763671.66970552
Login Successful.
Searching datasets...


HTTPError: 403 Client Error: Forbidden for url: https://m2m.cr.usgs.gov/api/api/json/stable/dataset-search